In [1]:
import os

# Changes the current path to find the source files

current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()

In [2]:
from utils import create_all_folders, Folders

create_all_folders()

### Get the cropping limits

In [3]:
from utils import create_folder, get_file_base_name
from geojson_conversions import open_geojson_feature_collection

from preprocessing.data import (
    find_annots_repartition,
    crop_annots_into_limits,
    annots_coordinates_to_local,
    save_annots_per_image,
    crop_all_images_from_annotations_folder,
    ImageData,
    get_cropping_limits,
    merge_tif,
)
from preprocessing.lidar import (
    download_lidar_names_shapefile,
    get_lidar_files_from_image,
    download_and_remove_overlap_geotiles,
    create_full_lidar,
)
from preprocessing.rgb_cir import download_rgb_image_from_polygon, download_cir
from preprocessing.chm import compute_slices_chm
import numpy as np

In [4]:
# Define tile size and OVERLAP
# TILE_SIZE = 1920  # Size of each tile
# OVERLAP = 480  # Overlap between tiles
TILE_SIZE = 640  # Size of each tile
OVERLAP = 0  # Overlap between tiles

In [5]:
# Annotations file to use:

annotations_file_name = "122000_484000.geojson"

In [6]:
annotations_path = os.path.join(Folders.FULL_ANNOTS.value, annotations_file_name)

annotations = open_geojson_feature_collection(annotations_path)

full_image_path_tif = download_rgb_image_from_polygon(annotations["bbox"])[0]

Download skipped: there is already a file at 'c:\Users\alexandreb\Documents\tree-segmentation\data\images\rgb\full\2023_122000_484000_RGB_hrl.tif'.


In [7]:
from typing import Sequence


image_data = ImageData(full_image_path_tif)

shapefile_path = download_lidar_names_shapefile()

GEOTILES_OVERLAP = 20
intersection_file_names = get_lidar_files_from_image(image_data, shapefile_path, GEOTILES_OVERLAP)
intersection_file_paths = download_and_remove_overlap_geotiles(
    intersection_file_names, GEOTILES_OVERLAP
)

full_lidar_path, full_lidar_filtered_path = create_full_lidar(intersection_file_paths, image_data)

RESOLUTION = 0.08

resolution = RESOLUTION

z_tops: Sequence[float] = [1, 2, 3, 5, 7, 10, 15, 20, np.inf]
z_limits_list = [(-np.inf, z_top) for z_top in z_tops]
full_chm_filtered_paths = [
    os.path.join(
        Folders.CHM.value,
        f"{round(resolution*100)}cm",
        "filtered",
        f"{str(bottom)}_{str(top)}",
        "full",
        f"{image_data.coord_name}.tif",
    )
    for (bottom, top) in z_limits_list
]
for folder_path in full_chm_filtered_paths:
    create_folder(os.path.dirname(folder_path))
compute_slices_chm(
    laz_file_name=full_lidar_path,
    output_tif_paths=full_chm_filtered_paths,
    resolution=resolution,
    z_limits_list=z_limits_list,
    skip_if_file_exists=True,
)

cir_full_image_path = os.path.join(Folders.FULL_CIR_IMAGES.value, f"{image_data.base_name}.tif")
download_cir(
    image_coords_box=image_data.coord_box,
    resolution=resolution,
    skip_if_file_exists=True,
    save_path=cir_full_image_path,
)

full_images_folders_paths = [
    os.path.dirname(folder_path) for folder_path in full_chm_filtered_paths
]
full_images_folders_paths.append(os.path.dirname(image_data.path))
full_images_folders_paths.append(os.path.dirname(cir_full_image_path))

Download skipped: there is already a file at 'c:\Users\alexandreb\Documents\tree-segmentation\data\lidar\geotiles\25GN1_13.LAZ'.
Download skipped: there is already a file at 'c:\Users\alexandreb\Documents\tree-segmentation\data\lidar\geotiles\25GN1_18.LAZ'.
Running compute_slices_chm... Execution time: 0.0000 seconds
Running download_cir... Execution time: 0.0000 seconds


In [8]:
from utils import get_files_in_folders


cropping_limits_x, cropping_limits_y = get_cropping_limits(full_image_path_tif, TILE_SIZE, OVERLAP)

visibility_threshold = 0.2
annots_repartition = find_annots_repartition(
    cropping_limits_x, cropping_limits_y, annotations, image_data, visibility_threshold
)

crop_annots_into_limits(annots_repartition)
annots_coordinates_to_local(annots_repartition)


output_image_prefix = get_file_base_name(full_image_path_tif)

annotations_output_directory = os.path.join(Folders.CROPPED_ANNOTS.value, output_image_prefix)
save_annots_per_image(
    annots_repartition,

    annotations_output_directory,
    full_image_path_tif,

    clear_if_not_empty=True,
)



images_output_directory = os.path.join(Folders.CROPPED_RGB_IMAGES.value, output_image_prefix)
full_images_paths = get_files_in_folders(full_images_folders_paths)
output_cropped_images_folders_paths = [
    os.path.splitext(full_image_path.replace("full", "cropped"))[0]
    for full_image_path in full_images_paths
]


cropped_images_folders_paths = crop_all_images_from_annotations_folder(
    annotations_output_directory,
    output_cropped_images_folders_paths=output_cropped_images_folders_paths,
    full_images_folders_paths=full_images_folders_paths,
    clear_if_not_empty=False,
    remove_unused=True,
)



cropped_chms_folders_paths = [
    folder_path for folder_path in cropped_images_folders_paths if Folders.CHM.value in folder_path
]


merge_tif(cropped_chms_folders_paths)


cropped_rgb_cir_folders_paths = [
    folder_path
    for folder_path in cropped_images_folders_paths

    if Folders.IMAGES.value in folder_path
]


merge_tif(cropped_rgb_cir_folders_paths)

c:\Users\alexandreb\AppData\Local\miniconda3\envs\tree-segment\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (156250000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
